In [1]:
import torch
import sys
sys.path.insert(0, './code')
from sqlnet.utils import *
from sqlnet.model.sqlnet import SQLNet
import argparse

In [2]:
n_word=768
bert_path = 'code/chinese_L-12_H-768_A-12'

In [3]:
dev_sql, dev_table, dev_db, test_sql, test_table, test_db = load_dataset(use_small=False, mode='test')

Loading dataset
Loaded 4396 data from data/val/val.json
Loaded 1197 data from data/val/val.tables.json
Loaded 4086 data from data/test/test.json
Loaded 1102 data from data/test/test.tables.json


In [4]:
model_bert, tokenizer, bert_config = get_bert(bert_path)
model = SQLNet(N_word=n_word, use_ca=True, gpu=True, trainable_emb=False)
model_path = 'saved_model/best_model'
print ("Loading from %s" % model_path)
model.load_state_dict(torch.load(model_path,map_location='cpu'))
print ("Loaded model from %s" % model_path)
model.eval()

vocab size:  21128
hidden_size:  768
num_hidden_layer:  12
num_attention_heads:  12
hidden_act:  gelu
intermediate_size:  3072
hidden_dropout_prob:  0.1
attention_probs_dropout_prob:  0.1
max_position_embeddings:  512
type_vocab_size:  2
initializer_range:  0.02
Load pre-trained parameters.
Using column attention on select number predicting
Using column attention on selection predicting
Using column attention on aggregator predicting
Using column attention on where predicting
Using column attention on where relation predicting
Loading from saved_model/best_model
Loaded model from saved_model/best_model


SQLNet(
  (sel_num): SelNumPredictor(
    (sel_num_lstm): LSTM(768, 384, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (sel_num_att): Linear(in_features=768, out_features=1, bias=True)
    (sel_num_col_att): Linear(in_features=768, out_features=1, bias=True)
    (sel_num_out): Sequential(
      (0): Linear(in_features=768, out_features=768, bias=True)
      (1): Tanh()
      (2): Linear(in_features=768, out_features=4, bias=True)
    )
    (softmax): Softmax(dim=-1)
    (sel_num_col2hid1): Linear(in_features=768, out_features=1536, bias=True)
    (sel_num_col2hid2): Linear(in_features=768, out_features=1536, bias=True)
  )
  (sel_pred): SelPredictor(
    (sel_lstm): LSTM(768, 384, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (sel_att): Linear(in_features=768, out_features=768, bias=True)
    (sel_col_name_enc): LSTM(768, 384, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
    (sel_out_K): Linear(in_features=768, out_featu

In [5]:
sql = dev_sql[0]
predict_input(model_bert, tokenizer, model, 1, sql, dev_table[sql['table_id']]['header'])

{'sel': [1],
 'agg': [0],
 'cond_conn_op': 1,
 'conds': [[0, 2, '长沙'], [3, 2, '3.17']]}